# ETL (Ingeniería de Datos)

### Importar Librerías

In [19]:
import pandas as pd
import sys
import os
import numpy as np
import warnings
import ast
import re

# Ignorar todas las advertencias
warnings.filterwarnings("ignore")

In [20]:
# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Navegar hacia el directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Agregar la ruta del proyecto al sys.path
sys.path.append(project_root)

Importar funciones para ETL (funciones creadas en la carpeta function, archivo ETL)

In [21]:
from functions.ETL import load_data, normalize, export # (funciones creadas en la carpeta function, archivo ETL)

## Extracción de datos

Ruta del archivo:

In [22]:
path = r'..\data\steam_games.json.gz'

Extracción y Visualización de datos

In [23]:
df = load_data(path) # (funciones creadas en la carpeta function, archivo ETL)
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


## Transformación de los Datos

### Limpieza de los datos

#### Eliminar Filas innecesarias

Eliminar filas con todos los elementos vacíos:

In [24]:
df_clean = df.dropna(how='all')
len(df_clean)

32135

app_name tiene dos elementos vacíos que respecto a otras columnas lo hace imposible de identificar un registro

In [25]:
df_clean = df_clean.dropna(subset=['app_name'])
len(df_clean)

32133

Identificar duplicados por id y app_name:

In [26]:
df_clean[df_clean.duplicated(df_clean.columns[df_clean.columns.isin(['app_name', 'id'])])]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


Eliminar duplicados

In [27]:
df_clean = df_clean.drop_duplicates(df_clean.columns[df_clean.columns.isin(['app_name', 'id'])], keep='first')
len(df_clean)

32132

Comparar columnas app_name con title:

In [28]:
distincts = df_clean[df_clean['app_name'] != df_clean['title']]
len(distincts)

2603

In [29]:
# Ver la diferencia entre los valores diferentes entre title y app_name
len(distincts) - distincts['title'].isnull().sum()

np.int64(555)

Las filas que no coinciden entre app_name y title se deben eliminar ya que son datos inconsistentes y no se puede determinar el nombre del juego. Existen más de 2000 valores nulos en la columna title y 555 que no coinciden con la columna app_name que sí está completa. Visualizando el contenido se puede observar que la diferencia se encientra en el caracter & que en la columna app_name se muestra normalmente mientras que en la columna title se muestra $amp;

In [30]:
filtered = distincts[distincts['title'].notnull()]
filtered.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88390,Telltale Games,"[Action, Adventure]",Sam & Max 101: Culture Shock,Sam &amp; Max 101: Culture Shock,http://store.steampowered.com/app/8200/Sam__Ma...,2006-10-17,"[Point & Click, Comedy, Adventure, Detective, ...",http://steamcommunity.com/app/8200/reviews/?br...,[Single-player],19.99,0.0,8200.0,Telltale Games
88393,Telltale Games,"[Action, Adventure]",Sam & Max 102: Situation: Comedy,Sam &amp; Max 102: Situation: Comedy,http://store.steampowered.com/app/8210/Sam__Ma...,2006-12-20,"[Adventure, Action]",http://steamcommunity.com/app/8210/reviews/?br...,[Single-player],19.99,0.0,8210.0,Telltale Games
88419,Electronic Arts,[Strategy],Command & Conquer: Red Alert 3,Command &amp; Conquer: Red Alert 3,http://store.steampowered.com/app/17480/Comman...,2008-10-28,"[Strategy, RTS, Base Building, Multiplayer, Co...",http://steamcommunity.com/app/17480/reviews/?b...,[Single-player],19.99,0.0,17480.0,EA Los Angeles


#### Eliminar columnas innecesarias reordenándolas:

In [31]:
# Se listan las listas a conservar de manera ordenada y se filtran
cols = ['id', 
        'app_name', 
        'genres', 
        'tags', 
        'specs', 
        'url', 
        'reviews_url', 
        'price',  
        'release_date', 
        'developer', 
        'publisher']

df_games = df_clean[cols]

In [32]:
df_games.head(3)

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher
88310,761140.0,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,2018-01-04,Kotoshiro,Kotoshiro
88311,643980.0,Ironbound,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,Free To Play,2018-01-04,Secret Level SRL,"Making Fun, Inc."
88312,670290.0,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,Free to Play,2017-07-24,Poolians.com,Poolians.com


#### Imputar valores nulos:

In [36]:
print(len(df_games))
df_games.isnull().sum()

22529


id              0
app_name        0
genres          0
tags            0
specs           0
url             0
reviews_url     0
price           0
release_date    0
developer       0
publisher       0
dtype: int64

Las columnas genres, tags specs tienen datos similares por lo que pueden unirse, llevar a lowercase y dividirse según los elementos.

In [35]:
# eliminar nulos para render
df_games.dropna(inplace=True)

In [37]:
# Rellenar los valores NaN con listas vacías
df_games['genres'] = df_games['genres'].apply(lambda x: x if isinstance(x, list) else [])
df_games['tags'] = df_games['tags'].apply(lambda x: x if isinstance(x, list) else [])
df_games['specs'] = df_games['specs'].apply(lambda x: x if isinstance(x, list) else [])
df_games.head(3)

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher
88310,761140.0,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,2018-01-04,Kotoshiro,Kotoshiro
88311,643980.0,Ironbound,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,Free To Play,2018-01-04,Secret Level SRL,"Making Fun, Inc."
88312,670290.0,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,Free to Play,2017-07-24,Poolians.com,Poolians.com


In [38]:
# Concatenar las listas de las columnas 'genres', 'tags' y 'specs' fila por fila
df_games['genres'] = df_games.apply(lambda row: row['genres'] + row['tags'] + row['specs'], axis=1)

# Ver el resultado
df_games.head(3)

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher
88310,761140.0,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy, ...","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,2018-01-04,Kotoshiro,Kotoshiro
88311,643980.0,Ironbound,"[Free to Play, Indie, RPG, Strategy, Free to P...","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,Free To Play,2018-01-04,Secret Level SRL,"Making Fun, Inc."
88312,670290.0,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Spor...","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,Free to Play,2017-07-24,Poolians.com,Poolians.com


In [39]:
# Convertir los elementos de cada lista en las columnas 'genres' y specs a minúsculas
df_games['genres'] = df_games['genres'].apply(lambda genres: [g.lower() for g in genres])
df_games['specs'] = df_games['specs'].apply(lambda genres: [g.lower() for g in genres])

# Ver el resultado
df_games.head(3)

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher
88310,761140.0,Lost Summoner Kitty,"[action, casual, indie, simulation, strategy, ...","[Strategy, Action, Indie, Casual, Simulation]",[single-player],http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,2018-01-04,Kotoshiro,Kotoshiro
88311,643980.0,Ironbound,"[free to play, indie, rpg, strategy, free to p...","[Free to Play, Strategy, Indie, RPG, Card Game...","[single-player, multi-player, online multi-pla...",http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,Free To Play,2018-01-04,Secret Level SRL,"Making Fun, Inc."
88312,670290.0,Real Pool 3D - Poolians,"[casual, free to play, indie, simulation, spor...","[Free to Play, Simulation, Sports, Casual, Ind...","[single-player, multi-player, online multi-pla...",http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,Free to Play,2017-07-24,Poolians.com,Poolians.com


In [40]:
# Filtrar los elementos de 'genres' eliminando los elementos con la palabra player
df_games['genres'] = df_games['genres'].apply(lambda specs: [s for s in specs if 'player' not in s])
# Filtrar los elementos de 'specs' que contengan la palabra 'player'
df_games['specs'] = df_games['specs'].apply(lambda specs: [s for s in specs if 'player' in s])

# Ver el resultado
df_games.head(3)

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher
88310,761140.0,Lost Summoner Kitty,"[action, casual, indie, simulation, strategy, ...","[Strategy, Action, Indie, Casual, Simulation]",[single-player],http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,2018-01-04,Kotoshiro,Kotoshiro
88311,643980.0,Ironbound,"[free to play, indie, rpg, strategy, free to p...","[Free to Play, Strategy, Indie, RPG, Card Game...","[single-player, multi-player, online multi-pla...",http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,Free To Play,2018-01-04,Secret Level SRL,"Making Fun, Inc."
88312,670290.0,Real Pool 3D - Poolians,"[casual, free to play, indie, simulation, spor...","[Free to Play, Simulation, Sports, Casual, Ind...","[single-player, multi-player, online multi-pla...",http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,Free to Play,2017-07-24,Poolians.com,Poolians.com


In [41]:
# Eliminar duplicados en cada lista de la columna 'genres' y 'specs'
df_games['genres'] = df_games['genres'].apply(lambda genres: list(set(genres)))
df_games['specs'] = df_games['specs'].apply(lambda genres: list(set(genres)))

# Ver el resultado
df_games.head(3)

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher
88310,761140.0,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]","[Strategy, Action, Indie, Casual, Simulation]",[single-player],http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,2018-01-04,Kotoshiro,Kotoshiro
88311,643980.0,Ironbound,"[steam trading cards, turn-based, fantasy, com...","[Free to Play, Strategy, Indie, RPG, Card Game...","[online multi-player, multi-player, cross-plat...",http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,Free To Play,2018-01-04,Secret Level SRL,"Making Fun, Inc."
88312,670290.0,Real Pool 3D - Poolians,"[sports, indie, casual, in-app purchases, stat...","[Free to Play, Simulation, Sports, Casual, Ind...","[online multi-player, multi-player, single-pla...",http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,Free to Play,2017-07-24,Poolians.com,Poolians.com


Las columnas developer y publisher suelen coincidir en algunos casos, por lo que pueden usarse para rellenar valores nulos.

In [42]:
df_games['developer'].fillna(df_games['publisher'], inplace=True)
df_games['developer'].isnull().sum()

np.int64(0)

#### Imputar columna price en caso de existir el valor 'free to play' en genres

In [43]:
# Rellenar los valores vacíos en 'price' con 0 si en 'genres' está 'free to play'
df_games['price'] = df_games.apply(
    lambda row: 0 if pd.isna(row['price']) and 'free to play' in row['genres'] else row['price'], 
    axis=1
)

# Ver el resultado
df_games.head(3)

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher
88310,761140.0,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]","[Strategy, Action, Indie, Casual, Simulation]",[single-player],http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,2018-01-04,Kotoshiro,Kotoshiro
88311,643980.0,Ironbound,"[steam trading cards, turn-based, fantasy, com...","[Free to Play, Strategy, Indie, RPG, Card Game...","[online multi-player, multi-player, cross-plat...",http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,Free To Play,2018-01-04,Secret Level SRL,"Making Fun, Inc."
88312,670290.0,Real Pool 3D - Poolians,"[sports, indie, casual, in-app purchases, stat...","[Free to Play, Simulation, Sports, Casual, Ind...","[online multi-player, multi-player, single-pla...",http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,Free to Play,2017-07-24,Poolians.com,Poolians.com


In [44]:
df_games['price'].isnull().sum()

np.int64(0)

In [45]:
# Eliminar "free to play" de las listas en la columna 'genres'
df_games['genres'] = df_games['genres'].apply(lambda x: [genre for genre in x if genre != 'free to play'])

# Ver el resultado
df_games.head(3)

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher
88310,761140.0,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]","[Strategy, Action, Indie, Casual, Simulation]",[single-player],http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,2018-01-04,Kotoshiro,Kotoshiro
88311,643980.0,Ironbound,"[steam trading cards, turn-based, fantasy, com...","[Free to Play, Strategy, Indie, RPG, Card Game...","[online multi-player, multi-player, cross-plat...",http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,Free To Play,2018-01-04,Secret Level SRL,"Making Fun, Inc."
88312,670290.0,Real Pool 3D - Poolians,"[sports, indie, casual, in-app purchases, stat...","[Free to Play, Simulation, Sports, Casual, Ind...","[online multi-player, multi-player, single-pla...",http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,Free to Play,2017-07-24,Poolians.com,Poolians.com


Identificar id nulo

In [46]:
df_games[df_games['id'].isnull()]

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher


In [47]:
# en la URL figura el id, por lo que se verifica si el registro ya existe
df_games[df_games['id'] == 200260]

,id,app_name,genres,tags,specs,url,reviews_url,price,release_date,developer,publisher
89378,200260.0,Batman: Arkham City - Game of the Year Edition,"[stealth, batman, metroidvania, steam trading ...","[Action, Open World, Batman, Adventure, Stealt...",[single-player],http://store.steampowered.com/app/200260/Batma...,http://steamcommunity.com/app/200260/reviews/?...,19.99,2012-09-07,"Rocksteady Studios,Feral Interactive (Mac)","Warner Bros. Interactive Entertainment, Feral ..."


In [48]:
# se eliminan los datos con id nulos tras verificar duplicidad
df_games = df_games.dropna(subset='id')
# se cambia el tipo de dato de la columna
df_games['id'] = df_games['id'].astype('int64')
df_games['id'].dtype

dtype('int64')

In [49]:
#volver a verificar datos nulos
df_games.isnull().sum()

id              0
app_name        0
genres          0
tags            0
specs           0
url             0
reviews_url     0
price           0
release_date    0
developer       0
publisher       0
dtype: int64

#### Se crea un DataFrame con las URLs de los juegos y las reviews
- Las URLs de los juegos pueden necesitarse para mejorar el sistema recomendando las páginas de los juegos.
- Las URLs de las reviews pueden utilizarse a futuro para Web Scraping

In [50]:
games_urls = df_games[['id', 'url', 'reviews_url']]

Eliminar columnas tags, publisher, url y url_reviews

In [51]:
# Eliminar columnas innecesarias y visualizar
df_steam = df_games.drop(columns=['tags', 'publisher', 'url', 'reviews_url'])
df_steam.head(3)

,id,app_name,genres,specs,price,release_date,developer
88310,761140,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]",[single-player],4.99,2018-01-04,Kotoshiro
88311,643980,Ironbound,"[steam trading cards, turn-based, fantasy, com...","[online multi-player, multi-player, cross-plat...",Free To Play,2018-01-04,Secret Level SRL
88312,670290,Real Pool 3D - Poolians,"[sports, indie, casual, in-app purchases, stat...","[online multi-player, multi-player, single-pla...",Free to Play,2017-07-24,Poolians.com


Comprobar tipos de datos

In [52]:
[print(c, df_steam[c].dtype) for c in df_steam.columns]

id int64
app_name object
genres object
specs object
price object
release_date object
developer object


[None, None, None, None, None, None, None]

#### Convertir los demás datos

In [53]:
#se convierten los textos a formato str
df_steam = df_steam.astype({'app_name': 'str', 'price': 'str', 'developer': 'str'})
[print(c, df_steam[c].dtype) for c in df_steam.columns]

id int64
app_name object
genres object
specs object
price object
release_date object
developer object


[None, None, None, None, None, None, None]

In [54]:
# Definir función para manejar los precios
# si incluyen la palabra Free, el precio se configura en 0
def handle_price(price):
    '''
    Si el precio contiene la palabra Free o free retorna 0.0,
    sino, intenta convertirlo a float y, en caso de no poder, retorna NaN
    '''
    if 'Free' in price or 'free' in price: 
        return 0.0
    # Si no contiene la palabra Free, intenta convertirlo a float y, en caso de no poder, retorna NaN
    try:
        return float(price)
    except ValueError:
        return np.nan

In [55]:
# se cambian los valores que implican Free to Play a Free
df_steam['price'] = df_steam['price'].apply(handle_price)
# verificar tipo de dato de la columna price

df_steam['price'].dtype

dtype('float64')

In [56]:
df_steam.head(3)

,id,app_name,genres,specs,price,release_date,developer
88310,761140,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]",[single-player],4.99,2018-01-04,Kotoshiro
88311,643980,Ironbound,"[steam trading cards, turn-based, fantasy, com...","[online multi-player, multi-player, cross-plat...",0.00,2018-01-04,Secret Level SRL
88312,670290,Real Pool 3D - Poolians,"[sports, indie, casual, in-app purchases, stat...","[online multi-player, multi-player, single-pla...",0.00,2017-07-24,Poolians.com


In [57]:
# Convertir las fechas en diferentes formatos a un formato unificado de fecha
df_steam['release_date'] = pd.to_datetime(df_steam['release_date'], errors='coerce')

# Extraer el año de las fechas convertidas
df_steam['release_year'] = df_steam['release_date'].dt.year

# Mostrar un ejemplo de las primeras filas
df_steam.head()

,id,app_name,genres,specs,price,release_date,developer,release_year
88310,761140,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]",[single-player],4.99,2018-01-04,Kotoshiro,2018.0
88311,643980,Ironbound,"[steam trading cards, turn-based, fantasy, com...","[online multi-player, multi-player, cross-plat...",0.00,2018-01-04,Secret Level SRL,2018.0
88312,670290,Real Pool 3D - Poolians,"[sports, indie, casual, in-app purchases, stat...","[online multi-player, multi-player, single-pla...",0.00,2017-07-24,Poolians.com,2017.0
88313,767400,弹炸人2222,"[adventure, action, casual]",[single-player],0.99,2017-12-07,彼岸领域,2017.0
88315,772540,Battle Royale Trainer,"[third-person shooter, steam achievements, sho...",[single-player],3.99,2018-01-04,Trickjump Games Ltd,2018.0


In [58]:
# eliminar columna release_date y mostrar los tres primeros resultados
df_steam = df_steam.drop(columns=['release_date'])
df_steam.head(3)

,id,app_name,genres,specs,price,developer,release_year
88310,761140,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]",[single-player],4.99,Kotoshiro,2018.0
88311,643980,Ironbound,"[steam trading cards, turn-based, fantasy, com...","[online multi-player, multi-player, cross-plat...",0.00,Secret Level SRL,2018.0
88312,670290,Real Pool 3D - Poolians,"[sports, indie, casual, in-app purchases, stat...","[online multi-player, multi-player, single-pla...",0.00,Poolians.com,2017.0


In [59]:
print(len(df_steam))
df_steam.isna().sum()

22529


id               0
app_name         0
genres           0
specs            0
price            5
developer        0
release_year    34
dtype: int64

In [60]:
# seguimos limpiando
df_games.dropna(inplace=True)

### tratamiento de los géneros

In [61]:
# Diccionario para reemplazar abreviaturas por nombres completos
genre_replacements = {
    'fps': 'first-person shooter',
    'first person shooter': 'first-person shooter',
    'first-person shooter': 'first-person shooter',
    'first person': 'first-person',
    'tps': 'third-person shooter',
    'third person': 'third-person',
    'third person shooter': 'third-person shooter',
    'third-person shooter': 'third-person shooter',
    'sim': 'simulation',
    'simulator': 'simulation',
    'rts': 'real-time strategy',
    'platform': 'platformer',
    'story rich': 'story-rich',
    'rpg': 'role-playing',
    'dungeon': 'role-playing',
    'shoot': 'shooter',
    'combat': 'fighting',
    'fiction': 'sci-fi',
    'real time': 'real-time',
    "shoot 'em up": "shoot'em up"
}

In [62]:
# Se define una función que agrega elementos a la lista géneros con valores normalizados
def replace_genres(genre_list):
    '''
    Recibe una lista de géneros, compara los elementos buscando substrings y agrega géneros acorde con lo que encontró.
    Si el valor es nulo o no es una lista, lo devuelve sin cambios.
    '''
    # Si el género es NaN (un solo valor NaN) o no es una lista
    if genre_list is None or (isinstance(genre_list, float) and pd.isna(genre_list)):
        return genre_list  # Devolvemos el valor sin cambios si es NaN
    
    if not isinstance(genre_list, list):
        return genre_list  # Si no es una lista, devolver sin cambios
    
    replaced_genres = list(genre_list)  # Clonamos la lista original para evitar modificarla directamente
    
    for genre in genre_list:
        # Recorremos el diccionario de reemplazos y agregamos el género correspondiente si el substring está presente
        for key, replacement in genre_replacements.items():
            if key in genre and replacement not in replaced_genres:  # Evitar duplicados
                replaced_genres.append(replacement)

    return replaced_genres  # Devolvemos la lista original más los géneros agregados

In [63]:
df_steam['genres'] = df_steam['genres'].apply(replace_genres)
df_steam.head()

,id,app_name,genres,specs,price,developer,release_year
88310,761140,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]",[single-player],4.99,Kotoshiro,2018.0
88311,643980,Ironbound,"[steam trading cards, turn-based, fantasy, com...","[online multi-player, multi-player, cross-plat...",0.00,Secret Level SRL,2018.0
88312,670290,Real Pool 3D - Poolians,"[sports, indie, casual, in-app purchases, stat...","[online multi-player, multi-player, single-pla...",0.00,Poolians.com,2017.0
88313,767400,弹炸人2222,"[adventure, action, casual]",[single-player],0.99,彼岸领域,2017.0
88315,772540,Battle Royale Trainer,"[third-person shooter, steam achievements, sho...",[single-player],3.99,Trickjump Games Ltd,2018.0


In [64]:
# lista con todos los géneros finales normalizados
genres = [
    'indie', 'action', 'adventure', 'casual', 'simulation', 'strategy', "co-op", 'puzzle', 'sports', 'online', 
    'platformer', "story rich", 'sci-fi', 'fantasy', 'horror', 'anime', 'shooter', 'racing', "first-person", 
    'local', 'sandbox', 'arcade', 'retro', "turn-based", 'mmo', 'comedy', 'survival', 'classic', 'gore', 
    "third-person", "visual novel", 'utilities', 'exploration', "hidden object", 'rogue-like', 'tactical', 
    'rpgmaker', 'design', 'illustration', 'physics', 'mystery', 'psychological', 'education', 'historical', 
    "fast-paced", 'building', "tower defense", 'relaxing', 'war', 'hack', 'slash', 'crafting', 'jrpg', 
    'animation', 'modeling', "party-based", 'fighting', 'rogue-lite', 'competitive', "role-playing", 'pvp', 'card', 
    "post-apocalyptic", 'board','cyberpunk', "turn-based", "top-down", 'metroidvania', 'drama', 'military', 'mmorpg', 
    'futuristic', 'city builder', 'flight', 'romance', 'trains', 'driving', 'dark', 'surreal', 'windows mixed reality', 
    'video', 'production', 'mature', 'steampunk', 'economy', "game development", "team-based", 'dating', 'audio', 
    'thriller', 'dark humor', "text-based", 'lovecraftian', 'arena', "real-time", 'clicker', 'otome', 'moba', 
    'gamemaker', 'mod', 'science', 'crpg', 'tactics', 'fmv', "e-sports", "god game", 'pve', 'hunting', 'mythology', 
    "martial arts", "class-based", 'agriculture', '6dof', "lore-rich", 'sailing', 'offroad', 'campaign', 'fishing', 
    'philisophical', 'mining', 'investigation', 'conversation', 'wrestling', 'nsfw', "shoot'em up"
]

In [65]:
# Se define una función que revisará todos los elementos de géneros y devolverá sólo los géneros normalizados
def new_genres(old_genres):
    '''
    Esta función recibe una lista con todos los géneros antiguos no normalizados, compara elemento por elemento buscando si existe un substring que coincida con los géneros finales para agregar estos últimos a una lista final que luego será retornada.
    '''
    # Lista para almacenar los géneros finales
    res = []

    # Si el género no es NaN (un solo valor NaN) y es una lista
    if old_genres is not None and not (isinstance(old_genres, float) and pd.isna(old_genres)):
        # recorremos cada elemento de la lista original
        for genre in old_genres:
            # se recorre la lista de géneros finales y agrega el género si el substring está presente
            for final_genre in genres:
                if final_genre in genre and final_genre not in res and 'steam ' not in genre: # evitar duplicados y que considere los tags con steam
                    res.append(final_genre)

    return res  # Devolvemos la lista de géneros finales

In [66]:
# se crea una nueva columna con los nuevos valores de género para compararlos
df_steam['new_genres'] = df_steam['genres'].apply(new_genres)

# Se compara la lista de géneros finales con la lista de géneros finales normalizados
df_steam.head(3)

,id,app_name,genres,specs,price,developer,release_year,new_genres
88310,761140,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]",[single-player],4.99,Kotoshiro,2018.0,"[indie, action, casual, simulation, strategy]"
88311,643980,Ironbound,"[steam trading cards, turn-based, fantasy, com...","[online multi-player, multi-player, cross-plat...",0.00,Secret Level SRL,2018.0,"[turn-based, fantasy, competitive, pvp, tactic..."
88312,670290,Real Pool 3D - Poolians,"[sports, indie, casual, in-app purchases, stat...","[online multi-player, multi-player, single-pla...",0.00,Poolians.com,2017.0,"[sports, indie, casual, simulation, strategy, ..."


In [67]:
# reemplazar generos por new_genres y eliminar la columna auxiliar new_genres
df_steam['genres'] = df_steam['new_genres']
df_steam = df_steam.drop(columns=['new_genres'])

#visualizar
df_steam.head(3)

,id,app_name,genres,specs,price,developer,release_year
88310,761140,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]",[single-player],4.99,Kotoshiro,2018.0
88311,643980,Ironbound,"[turn-based, fantasy, competitive, pvp, tactic...","[online multi-player, multi-player, cross-plat...",0.00,Secret Level SRL,2018.0
88312,670290,Real Pool 3D - Poolians,"[sports, indie, casual, simulation, strategy, ...","[online multi-player, multi-player, single-pla...",0.00,Poolians.com,2017.0


### Normalización de developers

In [68]:
norm_devs = {
    'raven software': 'raven soft',
    'ers g-studios': 'ers g',
    'ers game studios': 'ers g',
    'virtualrailroads': 'virtual railroads',
    'bitsalive': 'b alive',
    'spicyhorse': 'spicy horse',
    'forbes': 'forbes consult',
    'triangle service': 'triangle',
    'mercurysteam': 'mercury steam',
    'keen software house': 'keen',
    'bigant studios': 'big ant', 
    'io-interactive a/s': 'io interactive',
    'ortigames': 'ortisoft', 
    ' f/x': 'f x',
    'tt games': 'tt soft',
    ' a/s': ' a s',
    '1c-softclub': '1c,softclub'
}
#recordar, si no hay una coma antes del string o antes del ", " no considerar
#recordar que si una palabra es más larga y contiene un substring de una más corta, la segunda pierde validez (al menos si sigue con 'y' como trend, soft (que puede tener caracteres previos o posteriores))

In [69]:
# Se define una función para normalizar todos los desarrolladores
def devs_normalize(devs):
    ''' 
    Esta función desempeña las siguientes tareas:
    - Reemplaza los nombres de desarrolladores antiguos con los nuevos.
    - Elimina puntos, guiones o '&' reemplazándolos por ' '.
    - Elimina palabras innecesarias (&amp;, games, softwares, software, studios, studio, development, productions).
    - Separa los elementos con '/' o ',' si existen.
    - Elimina espacios en blanco de cada elemento.
    - Devuelve una lista vacía si no existen elementos o una lista con todos los elementos normalizados en su interior.
    '''
    # Lista para almacenar los desarrolladores normalizados
    res = []

    # Si el desarrollador no es nulo y es una cadena válida
    if isinstance(devs, str) and devs.strip():
        # se convierten los nombres a minúscula
        devs = devs.lower()

        # Se reemplazan los nombres de desarrolladores antiguos con los nuevos
        for old_dev, new_dev in norm_devs.items():
            devs = devs.replace(old_dev, new_dev)

        # Se eliminan puntos, guiones o '&' reemplazándolos por espacio
        devs = devs.replace('.', '').replace('-', '').replace('&amp;', ' ').replace('®', '').replace('&', ' ').replace('.com', '')

        # Elimina paréntesis y todo su contenido
        devs = re.sub(r'\(.*?\)', '', devs)

        # lista con palabras a eliminar
        remove = ['games', 'softwares', 'software', 'studios', 'studio', 'development', 'productions', 'interactives', 'interactive', 'entertainments', 'entertainment', 'designs', 'design', 'inc', 'montreal', 'romania', 'russia', 'ltd', 'srl', 'tecnologies', 'company', 'bulgaria', 'new england', 'san francisco', 'usa', 'llc', 'san diego', 'los angeles', 'technologies']

        # Se eliminan palabras innecesarias
        for word in remove:
            devs = re.sub(word, '', devs)

        # Se separan los elementos con '/' o ',' si existen, de lo contrario queda un solo nombre
        dev_list = re.split(r'[\/,:@]', devs)

        # Se eliminan espacios en blanco de cada elemento y solo añadimos si no es una cadena vacía
        dev_list = [d.strip() for d in dev_list if d.strip()]

        # Se eliminan espacios en blanco de cada elemento y los espacios internos también
        dev_list = [d.replace(' ', '') for d in dev_list if d.strip()]

        # Se añaden los elementos normalizados a la lista
        res.extend(dev_list)

    #se eliminan posibles datos duplicados
    res = list(set(res))
    return res  # Devolvemos la lista de desarrolladores normalizados


In [70]:
# Aplicamos la función de normalización de los desarrolladores - co
df_steam['new_devs'] = df_steam['developer'].apply(devs_normalize)
df_steam.head(3)

,id,app_name,genres,specs,price,developer,release_year,new_devs
88310,761140,Lost Summoner Kitty,"[indie, action, casual, simulation, strategy]",[single-player],4.99,Kotoshiro,2018.0,[kotoshiro]
88311,643980,Ironbound,"[turn-based, fantasy, competitive, pvp, tactic...","[online multi-player, multi-player, cross-plat...",0.00,Secret Level SRL,2018.0,[secretlevel]
88312,670290,Real Pool 3D - Poolians,"[sports, indie, casual, simulation, strategy, ...","[online multi-player, multi-player, single-pla...",0.00,Poolians.com,2017.0,[poolianscom]


In [71]:
# se reemplaza la columna developer por new_devs y esta última se elimina
df_steam['developer'] = df_steam['new_devs']
df_steam = df_steam.drop(columns=['new_devs'])

In [72]:
# desanidar los géneros, specs y developer
df_steam = df_steam.explode('genres')
df_steam = df_steam.explode('specs')
df_steam = df_steam.explode('developer')
df_steam.reset_index(drop=True, inplace=True)

#mostrar últimos cinco elementows
df_steam.tail()

,id,app_name,genres,specs,price,developer,release_year
161203,658870,EXIT 2 - Directions,casual,single-player,4.99,stev3ns,2017.0
161204,658870,EXIT 2 - Directions,puzzle,single-player,4.99,xropi,2017.0
161205,658870,EXIT 2 - Directions,puzzle,single-player,4.99,stev3ns,2017.0
161206,658870,EXIT 2 - Directions,relaxing,single-player,4.99,xropi,2017.0
161207,658870,EXIT 2 - Directions,relaxing,single-player,4.99,stev3ns,2017.0


## Carga de Datos

Se guarda el archivo trabajado en formato parquet y CSV en sus carpetas correspondientes para ser trabajados de acuerdo a la situación. Si los directorios no existen, se crean.

In [73]:
export(df_steam, project_root, 'steam_games')
# Se exporta, las URLs de los juegos y las reviews en caso de que a futuro se planee realizar web scraping
# descomentar siguiente línea para exportar archivos con URLs en caso de necesitarlo
#export(games_urls, project_root, 'games_urls')
# Función de exportación en ./functions/ETL.py

Archivos exportados exitosamente.
